# Feedfoward Neural Network in Pytorch
Coding a basic feedforward neural network in Pytorch. We will be building a CIFAR-10 classifier

In [ ]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

* Get the training and test data
* Convert the data from [0,1] to [-1,-1]

In [ ]:
def cifar_data(trainTrue: bool):
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
        ])
    out_dir = './dataset'
    return datasets.CIFAR10(root=out_dir, train=trainTrue, transform=compose, download=True)

train_data=cifar_data(trainTrue=True)
test_data=cifar_data(trainTrue=False)


Define the classes and the parameters for the training

In [ ]:
classes = ['plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck']
batch_size=128
random_seed=42

Create the dataloaders for the train and test set

In [ ]:
train_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size, shuffle=False)

Define the feedforward network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F 
class FFNet(nn.Module):
  def __init__(self):
    super(FFNet,self).__init__()
    self.fc0=nn.Linear(3*32*32, 1024)
    self.fc1=nn.Linear(1024,512)
    self.fc2=nn.Linear(512,128)
    self.fc3=nn.Linear(128,10)

  def forward(self, x):
    x = x.view(x.size(0), -1)
    x=self.fc0(x)
    x=self.fc1(x)
    x=self.fc2(x)
    x=self.fc3(x)
    return x

ff_net=FFNet()




Define loss function and an optimizer

For this case, we are using the Cross Entropy Loss and the SGD optimizer

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(ff_net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs=10
for epoch in range(num_epochs):
  running_loss=0.0

  for i, data in enumerate(train_loader, 0):

    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = ff_net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 100 == 99:    # print every 2000 mini-batches
        print('[Epoch %d, Iteration %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
dataiter=iter(test_loader)
images,labels=dataiter.next()
outputs=ff_net(images)
print(f1_score(torch.max(outputs.data,1)[-1],labels,average="micro"))
print(accuracy_score(torch.max(outputs.data,1)[-1],labels))


# Running it on GPU
We will now move this network to the GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Reinitialize train and test loaders

In [ ]:
train_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size, shuffle=False)

We need to move the following things from the CPU to the GPU
* The model
* The optimizer, since it uses the parameters from the pervious instantiation of the model
* The data i.e. inputs and labels

In [ ]:
ff_net=ff_net.to(device)
optimizer = optim.SGD(ff_net.parameters(), lr=0.001, momentum=0.9)
num_epochs=100
for epoch in range(num_epochs):
  running_loss=0.0

  for i, data in enumerate(train_loader, 0):

    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data[0].to(device), data[1].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = ff_net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 100 == 99:
        print('[Epoch %d, Iteration %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
dataiter=iter(test_loader)
images,labels=dataiter.next()
images,labels=images.to(device), labels.to(device)
outputs=ff_net(images)
print(f1_score(torch.max(outputs.data,1)[-1].cpu(),labels.cpu(),average="micro"))
print(accuracy_score(torch.max(outputs.data,1)[-1].cpu(),labels.cpu()))

# Making a CNN

Define the convolutional network

In [ ]:
import torch.nn.functional as F 
class ConvNet(nn.Module):
  
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

conv_net=ConvNet()
conv_net.to(device)

Reinitialize the train and test loaders

In [ ]:
train_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size, shuffle=False)

Define the loss and the optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(conv_net.parameters(), lr=0.0001)

In [ ]:
num_epochs=50
for epoch in range(num_epochs):
  running_loss=0.0

  for i, data in enumerate(train_loader, 0):

    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data[0].to(device), data[1].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = conv_net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 100 == 99:
        print('[Epoch %d, Iteration %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

Testing accuracy and F1-score on first test batch

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
dataiter=iter(test_loader)
images,labels=dataiter.next()
images,labels=images.to(device), labels.to(device)
outputs=conv_net(images)

print("F1 Score: ",f1_score(torch.max(outputs.data,1)[-1].cpu(),labels.cpu(),average="macro"))
print("Accuracy: ",accuracy_score(torch.max(outputs.data,1)[-1].cpu(),labels.cpu()))

Compute per-class accuracy

In [ ]:
class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))
for data in test_loader:
  images, labels = data
  images, labels = images.to(device), labels.to(device)
  outputs = conv_net(images)
  _, predicted = torch.max(outputs, 1)
  c = (predicted == labels).squeeze()
  for i in range(labels.size(0)):
    label = labels[i]
    class_correct[label] += c[i].item()
    class_total[label] += 1


for i in range(10):
  print('Accuracy of %5s : %2d %%' % (
    classes[i], 100 * class_correct[i] / class_total[i]))